In [5]:
import pandas as pd

df = pd.read_csv("./ds_salaries.csv")

display(df)

,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,2023,SE,FT,Principal Data Scientist,80000,EUR,85847,ES,100,ES,L
1,2023,MI,CT,ML Engineer,30000,USD,30000,US,100,US,S
2,2023,MI,CT,ML Engineer,25500,USD,25500,US,100,US,S
3,2023,SE,FT,Data Scientist,175000,USD,175000,CA,100,CA,M
4,2023,SE,FT,Data Scientist,120000,USD,120000,CA,100,CA,M
...,...,...,...,...,...,...,...,...,...,...,...
3750,2020,SE,FT,Data Scientist,412000,USD,412000,US,100,US,L
3751,2021,MI,FT,Principal Data Scientist,151000,USD,151000,US,100,US,L
3752,2020,EN,FT,Data Scientist,105000,USD,105000,US,100,US,S
3753,2020,EN,CT,Business Data Analyst,100000,USD,100000,US,100,US,L


In [6]:
df_features = df.copy(deep=True)

target = df_features["salary_in_usd"]
df_features.drop(["salary", "salary_in_usd"], axis=1, inplace=True)

display(df_features)

,work_year,experience_level,employment_type,job_title,salary_currency,employee_residence,remote_ratio,company_location,company_size
0,2023,SE,FT,Principal Data Scientist,EUR,ES,100,ES,L
1,2023,MI,CT,ML Engineer,USD,US,100,US,S
2,2023,MI,CT,ML Engineer,USD,US,100,US,S
3,2023,SE,FT,Data Scientist,USD,CA,100,CA,M
4,2023,SE,FT,Data Scientist,USD,CA,100,CA,M
...,...,...,...,...,...,...,...,...,...
3750,2020,SE,FT,Data Scientist,USD,US,100,US,L
3751,2021,MI,FT,Principal Data Scientist,USD,US,100,US,L
3752,2020,EN,FT,Data Scientist,USD,US,100,US,S
3753,2020,EN,CT,Business Data Analyst,USD,US,100,US,L


# Create preprocessor

In [7]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.feature_extraction.text import CountVectorizer
from nltk import word_tokenize

def define_job_type(job_title):
    if "Head" in job_title:
        return "Head"
    if "Director" in job_title:
        return "Director"
    if "Manager" in job_title:
        return "Manager"
    tokens = word_tokenize(job_title)
    return tokens[-1] 
def define_job_domain(job_title):
    if "Data" in job_title or "Analytics" in job_title:
        return "Data"
    if "Machine Learning" in job_title or "ML" in job_title or "AI" in job_title or "Deep Learning":
        return "ML"
    return job_title


df_features["job_type"] = df["job_title"].apply(define_job_type) 
df_features["job_domain"] = df["job_title"].apply(define_job_domain) 
df_features["job"] = df_features["job_type"] + df_features["job_domain"]

oneHot_features = ["work_year", "job", "employment_type", "remote_ratio", "salary_currency", "employee_residence", "company_location","company_size", "experience_level"]


print("Number of job titles before preprocessing:", df_features["job_title"].nunique())
print("Number of job titles before preprocessing:", df_features["job"].nunique())

preprocessor = ColumnTransformer(transformers=[
    ('cat', OneHotEncoder(), oneHot_features),
    ])
df_preprocessed = preprocessor.fit_transform(df_features)

display(pd.DataFrame(data=df_preprocessed.toarray(), columns=preprocessor.get_feature_names_out()).to_csv("test.csv"))

Number of job titles before preprocessing: 93
Number of job titles before preprocessing: 21


None

# Split data

In [8]:
from sklearn.model_selection import train_test_split

target_stratified = pd.cut(target, bins=6, labels=False)

print(target_stratified)

0       1
1       0
2       0
3       2
4       1
       ..
3750    5
3751    1
3752    1
3753    1
3754    1
Name: salary_in_usd, Length: 3755, dtype: int64


In [9]:
# Split into 80/20 Training / Testing
X_train, X_holdout, target_train, target_holdout = train_test_split(df_preprocessed, target, test_size=0.2, random_state=42)

X_train_stratified, X_holdout_stratified, target_train_stratified, target_holdout_stratified = train_test_split(df_preprocessed, target, test_size=0.2, stratify=target_stratified, random_state=42)

# Save data into npz

In [10]:
from scipy.sparse import save_npz

save_npz('./ds_salaries_Title_Clustering_Preprocessing_train.npz', X_train)
save_npz('./ds_salaries_Title_Clustering_Preprocessing_train_stratified.npz', X_train_stratified)

target_train.to_csv('./ds_salaries_target_train.csv', index=False)
target_train_stratified.to_csv('./ds_salaries_target_train_stratified.csv', index=False)


save_npz('./ds_salaries_Title_Clustering_Preprocessing_test.npz', X_holdout)
save_npz('./ds_salaries_Title_Clustering_Preprocessing_test_stratified.npz', X_holdout_stratified)

target_holdout.to_csv('./ds_salaries_target_test.csv', index=False)
target_holdout_stratified.to_csv('./ds_salaries_target_test_stratified.csv', index=False)

display(target_train.describe())
display(target_holdout.describe())


count      3004.000000
mean     138055.989348
std       63102.221140
min        5132.000000
25%       95000.000000
50%      135000.000000
75%      176250.000000
max      450000.000000
Name: salary_in_usd, dtype: float64

count       751.000000
mean     135627.992011
std       62873.313538
min        5409.000000
25%       95000.000000
50%      130000.000000
75%      170275.000000
max      385000.000000
Name: salary_in_usd, dtype: float64